# Notebook to check the calibration machinery.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Mandatory imports
%config InlineBackend.figure_format='retina'

%matplotlib widget

import os
# import glob
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import fitsio
from astropy.io import fits

from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qinstrument import QubicMultibandInstrument
from qubic.lib.Qscene import QubicScene

plt.rcParams['figure.figsize'] = (8,4)



In [ ]:
# Same names as in CreateSynthBeamCalFiles.ipynb but different functions, careful

def print_keys(d, keys):
    for key in keys:
        print(' - {:25}: {}'.format(key, d[key]))
    print('\n')



### This function retrieves the peaks information as a function of frequency, for each of the MultiBandInstrument sub-bands
def get_peaks_configuration(d, doplot=False, idet=None, debug=False):
    if d['config'] == 'TD':
        Ndet = 248
    elif d['config'] == 'FI':
        Ndet = 992
    else:
        print('Wrong config in dict')
        return 0
    
        print_keys(d, ['config', 'instrument_type', 'synthbeam', 'use_synthbeam_file', 'synthbeam_fraction', 'synthbeam_kmax'])
    # try:
    q_instrument = QubicMultibandInstrument(d)
    q_scene = QubicScene(d)
    print('done', config, instrument_type, 'nf_sub={}'.format(nf_sub))
    # except:
    #     print('oups ! failed instanciating QubicMultibandInstrument()')
    #     return 0,0,0,0
    
    n_nus = len(q_instrument)
    nus = np.zeros(n_nus)
    dnus = np.zeros(n_nus)
    print()
    print('The instrument has {} sub-frequencies'.format(n_nus))
    for i in range(n_nus):
        nus[i] = q_instrument.subinstruments[i].d['filter_nu']/1e9
        dnus[i] = q_instrument.subinstruments[i].d['filter_relative_bandwidth']*q_instrument.subinstruments[i].d['filter_nu']/1e9
        print('- {0:}: nu = {1:7.2f} GHz ; bw = {2:7.2f}'.format(i, nus[i], dnus[i]))
    
    thetas = np.zeros((n_nus, Ndet, (2*d['synthbeam_kmax']+1)**2))
    phis = np.zeros((n_nus, Ndet, (2*d['synthbeam_kmax']+1)**2))
    vals = np.zeros((n_nus, Ndet, (2*d['synthbeam_kmax']+1)**2))

    hdu = fits.open(q_instrument.subinstruments[i].calibration.synthbeam)

    for i in range(n_nus):
        position = q_instrument.subinstruments[i].detector.center
        # thetas[i,:,:], phis[i,:,:], vals[i,:,:] = q_instrument.subinstruments[i]._peak_angles(q_scene, 
        thetas[i,:,:], phis[i,:,:], vals[i,:,:] = q_instrument.subinstruments[i]._peak_angles_unsorted(q_scene, 
                                                    q_instrument.subinstruments[i].d['filter_nu'], 
                                                    q_instrument.subinstruments[i].detector.center, 
                                                    q_instrument.subinstruments[i].synthbeam, 
                                                    q_instrument.subinstruments[i].horn, 
                                                    q_instrument.subinstruments[i].primary_beam)
        
        
        if hdu[0].header['include_sec_beam']:
            theta = np.arctan2(np.sqrt(np.sum(position[..., :2] ** 2, axis=-1)), position[..., 2]) # this is code duplication from QubicInstrument._get_detector_integration_operator, not ideal!
            phi = np.arctan2(position[..., 1], position[..., 0])
            sec_beam_fact = q_instrument.subinstruments[i].secondary_beam(theta, phi)
            vals[i,:,:] *= sec_beam_fact[:, None] # this is needed to create a "realistic" calfile. Is it really realistic?

    thetas_fits = hdu[0].data
    phis_fits = hdu[1].data
    vals_fits = hdu[2].data
    nus_fits = hdu[3].data

    # print("thetas", thetas)
    # print("thetas_fits", thetas_fits)
    print("vals", vals)
    print("vals_fits", vals_fits)


    print("thetas == thetas_fits", np.all(thetas == thetas_fits))
    print("phis == phis_fits", np.all(phis == phis_fits))
    print("vals == vals_fits", np.all(vals == vals_fits))
    print("nus == nus_fits", np.all(nus == nus_fits))

    return thetas, phis, vals, nus, q_instrument

def update_dict(config, instrument_type, nf_sub, dictfilename='qubic/qubic/dicts/pipeline_demo.dict', debug=True):
    d = qubicDict()
    d.read_from_file(dictfilename)
    d['config'] = config
    
    d['instrument_type'] = instrument_type
    d['nf_sub'] = nf_sub
    d['debug'] = debug
    d['nside'] = 128              # To have nice SB maps

    d['beam_shape'] = 'gaussian'  # can be 'gaussian', 'fitted_beam' or 'multi_freq'  
    d['use_synthbeam_fits_file'] = True
    d['synthbeam_fraction'] = 1
    d['synthbeam_kmax'] = 1
    # d['synthbeam'] = "CalQubic_SynthBeam_TD_150.fits"         # we put the name of the file we created
    d['synthbeam'] = "CalQubic_SynthBeam_Analytical_nsideNNN_CC_FFF_kmaxK.fits"         # we put the name of the file we created
    print(d["synthbeam"])
    return d



In [ ]:
config = 'TD'
instrument_type = 'MB'
nf_sub = 10
d = update_dict(config, instrument_type, nf_sub)

idet = 140 #231
thetas, phis, vals, nus, qi = get_peaks_configuration(d,  
                                                      doplot=True, 
                                                      idet=idet)


In [ ]:
d["filter_nu"]

In [ ]:
qi[0]._get_projection_operator(None, None, d["filter_nu"], None, None, None, None, thetas, phis, vals, True, nus, interp_projection=False, verbose=True)

In [ ]:
qi.get_projection_operator(sampling, scene, verbose=True, interp_projection=False)